<a href="https://colab.research.google.com/github/arunsechergy/TSAI-DeepNLP-END/blob/main/assignments/assignment9/Assignment9_Model2_Attention_NarrativeQA_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this second notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq4.png?raw=1)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Preparing Data

All of the data preparation will be (almost) the same as last time, so we'll very briefly detail what each code block does. See the previous notebook for a recap.

We'll import PyTorch, TorchText, spaCy and a few standard modules.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, Example, Dataset

import spacy
import numpy as np

import random
import math
import time

Then set a random seed for deterministic results/reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Instantiate our German and English spaCy models.

Previously we reversed the source (German) sentence, however in the paper we are implementing they don't do this, so neither will we.

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [3]:
SRC = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = 'spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

Load our data.

In [4]:
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Read and preprocess the data

In [5]:
docs_index = pd.read_csv('https://raw.githubusercontent.com/deepmind/narrativeqa/master/documents.csv')
qapair = pd.read_csv('https://raw.githubusercontent.com/deepmind/narrativeqa/master/qaps.csv')

In [6]:
wiki_summary = pd.read_csv("https://raw.githubusercontent.com/deepmind/narrativeqa/master/third_party/wikipedia/summaries.csv")

In [7]:
wiki_summary.head(2)

,document_id,set,summary,summary_tokenized
0,0025577043f5090cd603c6aea60f26e236195594,test,"Mark Hunter (Slater), a high school student i...","Mark Hunter ( Slater ) , a high school student..."
1,0029bdbe75423337b551e42bb31f9a102785376f,train,"At Madeline Hall, an old mansion-house near S...","At Madeline Hall , an old mansion-house near S..."


In [8]:
docs_index.head(2)

,document_id,set,kind,story_url,story_file_size,wiki_url,wiki_title,story_word_count,story_start,story_end
0,0025577043f5090cd603c6aea60f26e236195594,test,movie,http://www.awesomefilm.com/script/pumpupthevol...,54078,http://en.wikipedia.org/wiki/Pump_Up_the_Volum...,Pump Up the Volume (film),11499,Happy Harry Hardon,by Martin Eaves
1,0029bdbe75423337b551e42bb31f9a102785376f,train,gutenberg,http://www.gutenberg.org/ebooks/21572.txt.utf-8,814507,http://en.wikipedia.org/wiki/Percival_Keene,Percival Keene,173334,Produced by Nick,new eBooks .


In [9]:
qapair

,document_id,set,question,answer1,answer2,question_tokenized,answer1_tokenized,answer2_tokenized
0,0025577043f5090cd603c6aea60f26e236195594,test,Who is Mark Hunter?,He is a high school student in Phoenix.,A loner and outsider student with a radio stat...,Who is Mark Hunter ?,He is a high school student in Phoenix .,A loner and outsider student with a radio stat...
1,0025577043f5090cd603c6aea60f26e236195594,test,Where does this radio station take place?,It takes place in Mark's parents basement.,"Phoenix, Arizona",Where does this radio station take place ?,It takes place in Mark s parents basement .,"Phoenix , Arizona"
2,0025577043f5090cd603c6aea60f26e236195594,test,Why do more students tune into Mark's show?,Mark talks about what goes on at school and in...,Because he has a thing to say about what is ha...,Why do more students tune into Mark s show ?,Mark talks about what goes on at school and in...,Because he has a thing to say about what is ha...
3,0025577043f5090cd603c6aea60f26e236195594,test,Who commits suicide?,Malcolm.,Malcolm.,Who commits suicide ?,Malcolm .,Malcolm .
4,0025577043f5090cd603c6aea60f26e236195594,test,What does Paige jam into her microwave?,She jams her medals and accolades.,Her award medals,What does Paige jam into her microwave ?,She jams her medals and accolades .,Her award medals
...,...,...,...,...,...,...,...,...
46760,ffcf7daee9cda766d2fcf1f6399b29be41876b21,test,What does the real Evelyn Mulwray do when she ...,Threatens to sue him.,she files a lawsuit against him,What does the real Evelyn Mulwray do when she ...,Threatens to sue him .,she files a lawsuit against him
46761,ffcf7daee9cda766d2fcf1f6399b29be41876b21,test,Who is the protagonist in this story?,Jake.,P.I. Jake Gittes,Who is the protagonist in this story ?,Jake .,P.I . Jake Gittes
46762,ffcf7daee9cda766d2fcf1f6399b29be41876b21,test,Who is the antagonist in this story?,Evelyn Mulwray.,Evelyn.,Who is the antagonist in this story ?,Evelyn Mulwray .,Evelyn .
46763,ffcf7daee9cda766d2fcf1f6399b29be41876b21,test,Where was Hollis Mulwray murdered?,In the salt water pond at his house.,freshwater resevoir,Where was Hollis Mulwray murdered ?,In the salt water pond at his house .,freshwater resevoir


In [10]:
docs_index.shape, qapair.shape, wiki_summary.shape

((1572, 10), (46765, 8), (1572, 4))

In [11]:
qapair.columns, wiki_summary.columns

(Index(['document_id', 'set', 'question', 'answer1', 'answer2',
        'question_tokenized', 'answer1_tokenized', 'answer2_tokenized'],
       dtype='object'),
 Index(['document_id', 'set', 'summary', 'summary_tokenized'], dtype='object'))

In [12]:
mapped_ads = qapair.merge(wiki_summary[['document_id', 'summary']], on='document_id', how='left')

In [13]:
mapped_ads_filtered = mapped_ads[['question', 'answer1', 'summary', 'set']]

In [14]:
mapped_ads_filtered['question'][0]

'Who is Mark Hunter?'

In [15]:
mapped_ads_filtered['summary'][0][:400]

' Mark Hunter (Slater), a high school student in a sleepy suburb of Phoenix, Arizona, starts an FM pirate radio station that broadcasts from the basement of his parents\' house. Mark is a loner, an outsider, whose only outlet for his teenage angst and aggression is his unauthorized radio station. His pirate station\'s theme song is "Everybody Knows" by Leonard Cohen and there are glimpses of cassette'

In [16]:
MAX_LEN = 400
mapped_ads_filtered['summary_question'] = mapped_ads_filtered['summary'].apply(lambda x: x[:MAX_LEN]) + mapped_ads_filtered['question']
mapped_ads_filtered['answer'] = mapped_ads_filtered['answer1']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
len(mapped_ads_filtered['summary_question'][0])

419

In [18]:
mapped_ads_filtered[mapped_ads_filtered['summary'].isna()]

,question,answer1,summary,set,summary_question,answer


In [19]:
df= mapped_ads_filtered[['summary_question', 'answer', 'set']]

In [20]:
test_df, train_df, valid_df  = [group for _, group in df.groupby('set')]
test_df.reset_index(drop=True, inplace=True)
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [21]:
len(train_df['summary_question'][10])

448

In [22]:
# This pandas dataframe should be mapped to the list of fields object

In [23]:
# formed the list of fields object
fields = [('src',SRC), ('trg', TRG)]

In [24]:
# map operation to map the rows of pandas dataframe to torchdataset field
train_mapped_rows_to_torchtext = [Example.fromlist([train_df['summary_question'][i], train_df['answer'][i]], fields) for i in range(train_df.shape[0])]
test_mapped_rows_to_torchtext = [Example.fromlist([test_df['summary_question'][i], test_df['answer'][i]], fields) for i in range(test_df.shape[0])]
valid_mapped_rows_to_torchtext = [Example.fromlist([valid_df['summary_question'][i], valid_df['answer'][i]], fields) for i in range(valid_df.shape[0])]

In [25]:
train_data = Dataset(train_mapped_rows_to_torchtext, fields)
valid_data = Dataset(valid_mapped_rows_to_torchtext, fields)
test_data =  Dataset(test_mapped_rows_to_torchtext, fields)

We'll also print out an example just to double check they're not reversed.

In [26]:
print(vars(valid_data.examples[0]))

{'src': [' ', 'the', 'play', 'begins', 'with', 'three', 'pages', 'disputing', 'over', 'the', 'black', 'cloak', 'usually', 'worn', 'by', 'the', 'actor', 'who', 'delivers', 'the', 'prologue', '.', 'they', 'draw', 'lots', 'for', 'the', 'cloak', ',', 'and', 'one', 'of', 'the', 'losers', ',', 'anaides', ',', 'starts', 'telling', 'the', 'audience', 'what', 'happens', 'in', 'the', 'play', 'to', 'come', ';', 'the', 'others', 'try', 'to', 'suppress', 'him', ',', 'interrupting', 'him', 'and', 'putting', 'their', 'hands', 'over', 'his', 'mouth', '.', 'soon', 'they', 'are', 'fighting', 'over', 'the', 'cloak', 'and', 'criticizing', 'the', 'author', 'and', 'who', 'normally', 'delivers', 'the', 'opening', 'prologue', 'in', 'the', 'play', '?'], 'trg': ['the', 'actor', 'wearing', 'the', 'black', 'cloak']}


In [27]:
print(vars(train_data.examples[0]))

{'src': [' ', 'at', 'madeline', 'hall', ',', 'an', 'old', 'mansion', '-', 'house', 'near', 'southampton', 'belonging', 'to', 'the', 'wealthy', 'de', 'versely', 'family', ',', 'lives', 'an', 'elderly', 'spinster', 'miss', 'delmar', ',', 'the', 'aunt', 'of', 'the', 'earl', 'de', 'versely', 'and', 'captain', 'delmar', '.', 'miss', 'delmar', 'invites', 'arabella', 'mason', ',', 'the', 'daughter', 'of', 'a', 'deceased', ',', 'well', '-', 'liked', 'steward', 'to', 'stay', 'with', 'her', 'as', 'a', 'lower', '-', 'class', 'guest', 'in', 'the', 'house', '.', 'captain', 'delmar', 'is', 'known', 'to', 'visit', 'his', 'aunt', 'at', 'madeline', 'hall', 'frequently', ',', 'who', 'is', 'miss', 'delmer', '?'], 'trg': ['the', 'elderly', 'spinster', 'aunt', 'of', 'the', 'earl', 'de', 'verseley', 'and', 'captain', 'delmar']}


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [28]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

Finally, define the `device` and create our iterators.

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:

device

device(type='cuda')

In [30]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False)

In [32]:
print("dataload complete")

dataload complete


In [33]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [34]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [35]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [36]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [37]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [38]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19721, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(10695, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=10695, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [39]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 33,395,911 trainable parameters


In [40]:
optimizer = optim.Adam(model.parameters())

In [41]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [42]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [44]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [46]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 3m 18s
	Train Loss: 6.207 | Train PPL: 496.295
	 Val. Loss: 5.334 |  Val. PPL: 207.322
Epoch: 02 | Time: 3m 20s
	Train Loss: 5.559 | Train PPL: 259.498
	 Val. Loss: 5.327 |  Val. PPL: 205.755
Epoch: 03 | Time: 3m 20s
	Train Loss: 5.259 | Train PPL: 192.273
	 Val. Loss: 5.347 |  Val. PPL: 210.037
Epoch: 04 | Time: 3m 19s
	Train Loss: 4.889 | Train PPL: 132.864
	 Val. Loss: 5.371 |  Val. PPL: 215.063
Epoch: 05 | Time: 3m 22s
	Train Loss: 4.443 | Train PPL:  84.997
	 Val. Loss: 5.425 |  Val. PPL: 226.993
Epoch: 06 | Time: 3m 20s
	Train Loss: 4.002 | Train PPL:  54.709
	 Val. Loss: 5.532 |  Val. PPL: 252.572
Epoch: 07 | Time: 3m 21s
	Train Loss: 3.596 | Train PPL:  36.435
	 Val. Loss: 5.699 |  Val. PPL: 298.679
Epoch: 08 | Time: 3m 21s
	Train Loss: 3.239 | Train PPL:  25.504
	 Val. Loss: 5.819 |  Val. PPL: 336.469
Epoch: 09 | Time: 3m 21s
	Train Loss: 2.861 | Train PPL:  17.486
	 Val. Loss: 5.975 |  Val. PPL: 393.405
Epoch: 10 | Time: 3m 19s
	Train Loss: 2.535 | Train PPL

In [47]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.359 | Test PPL: 212.494 |


In [48]:
print("COMPLETED RUNNING ATTENTION ON THIS DATSET")

COMPLETED RUNNING ATTENTION ON THIS DATSET
